## Coalesce Memory Accesses

### Learning objectives

In this lab we will study the ideal way to access GPU memory in CUDA kernels. For the vast majority of kernels, this is the second most important performance concern, right after exposing sufficient parallelism. We will learn:

- How to maximized global memory throughput
- The nature of caching on NVIDIA GPUs and how it relates to performance

After completing this lab, you will be able to understand how to write your kernel in a way that maximizes memory throughput.

### Prerequisites

It is assumed that participants have familiarity with:

- How to launch CUDA kernels that use both blocks and threads


- Basic memory management (`cudaMalloc()`, `cudaFree()`, `cudaMemcpy`)


- How to compile and run CUDA code


- Selecting an optimal kernel launch configuration for exposing massive parallelism

## Memory Architecture

Generally speaking, all processors have some sort of memory stack or hierarchy, with smaller amounts of memory closer to the chip (that are faster, in terms of both latency and throughput), and larger amounts of memory farther away from the chip (that are slower).

![](images/memory_architecture.png)

The actual GPU chip is depicted in blue above. The die has several SMs, each with their own shared memory/L1 cache, as well as a device-wide L2 cache. There are other types of caches on the GPU, such as texture and constant memory, which we won't cover in this session but may become relevant to your performance work in the future.

The green resource, DRAM, is usually on the GPU board, but not on the GPU die itself. DRAM is the physical backing for the (logical) global memory space, and is actually also the backing for local memory as well. (For example, if we have too much thread-local data to store in registers, it will spill back to global memory and be cached like normal data, on most recent GPU architectures.) Shared memory is a logical space and is backed by physical space on the chip (on recent GPUs, a unified shared memory/L1 cache).

The host (CPU) is separate, usually connected to the GPU by an interconnect like PCIe or NVLink, and we use it to transfer data to and from the GPU.

## GPU Memory Hierarchy

Let's start with a discussion of the memory hierarchy of the GPU. A GPU can be thought of as a collection of SMs (cookie-cutter sections of the GPU die, and we put one or more of these down on the die to make a GPU, with more powerful GPUs having more SMs). Each SM has registers, L1 cache and shared memory resources. L2 is a device-wide resource -- it is a separate physical entity from the SMs, and each SM talks to L2. Then we have DRAM/off-chip memory.

![](images/memory_hierarchy.png)


### Local storage

Each thread has its own (logical) **local** storage space. The most important local storage resource is registers. Registers are high-speed memory storage within the GPU chip itself, and they are the source and destination of nearly all low-level machine instructions.

Register usage is usually managed by the compiler. It's not advised to try to micromanage which data is in which registers. Focus on writing good C/C++ code and let the compiler manage this.


### Shared memory / L1 cache

Thsee are also both on-chip resources on the GPU die itself. **Shared memory** is a local memory array that can be allocated explicitly by the programmer and used for temporary storage of data. Since it's on-chip, it's a fast source of data. On recent GPUs there are programmable characteristics of shared memory. All GPUs have a minimum of 48 kB of shared memory, and some can be configured to use more (e.g. 64 or 96 kB).

L1 cache is not an explicitly user-managed resource; rather, like all other caches it's designed to improve performance by retaining recently used data in the hope that it can be used again rather than returning to device memory.

Both are high throughput and relatively low latency resources. Typically aggegrate throughputs are much higher than available global memory throughput (i.e. much more than ~1 TB/s).


### L2 cache

This is also a cache similar to the L1 cache that stores recently accessed data. Unlike L1, which is a per-SM resource, L2 is a device-wide resource. All data goes through L2 before it gets to/from device memory.


### Global memory

Global memory is usually the first resource used by any CUDA programmer. Global memory / device memory is accessible by all GPU threads, as well as the CPU. It is the location where memory allocated with `cudaMalloc()` comes from.

Global memory has relatively high latency (measured in the hundreds of clock cycles) -- it takes a relatively long time for data to be loaded into registers. This latency can often be a performance limiter for our code; we need to hide this latency by switching to other available work, which requires us to have sufficient exposed parallelism.

Compared to the RAM available on most CPUs, GPU global memory has relatively high throughput (for example, ~1 TB/s on V100 and ~1.5 TB/s on A100), and this is one of the major benefits of using GPUs.

Note: technically global memory is a logical memory space, while device memory (RAM or DRAM) is a physical memory space. Often these two terms are used interchangeably, because accesses to global memory typically translate to accesses to device memory, and we will mostly use these terms that way in this lecture. There are cases where the distinction becomes relevant which we will not focus on right now.

## Exercise: Caching and Performance

It is simple to experimentally observe how caches affect simple memory access patterns. In fact, a vector addition example is good enough to demonstrate this. All we need to do is run the same kernel multiple times in a row. Since this kernel is memory bandwidth bound, if the kernel is faster in the subsequent iterations compared to the first, this is an indirect but clear indication that the first kernel effectively preloaded the data into cache memory, and subsequent kernel launches read the data from cache memory instead of device (global) memory.

To that end, we've provided in [exercises/vector_add.cu](exercises/vector_add.cu) a vector addition example that is similar to the one we've seen before, but this time we run the same kernel 5 times in a row. Let's profile the code with Nsight Compute and see what it says about kernel duration. We can collect specific performance counters with Nsight Compute using the `--metrics` option. The metric `sm__cycles_elapsed.sum` measures the number of clock cycles that elapse during the kernel; more clock cycles means longer duration. So our hypothesis is that the number of cycles is longer in the first iteration than in subsequent iterations.

Your exercise is to vary the `DSIZE` parameter; try making it smaller (or larger) than the current value and see what happens. A typical L2 cache size on a modern NVIDIA GPU is O(10 MB), so we might expect caching benefits for arrays smaller than that size, and little benefit for arrays much larger than that. Are your results consistent with that hypothesis?

(Note: we are using the `--cache-control none` option to `ncu` to tell it not to interfere with the normal caching behavior of the GPU. For these experiments we don't want that.)

In [ ]:
!nvcc -arch=native -o vector_add exercises/vector_add.cu; ncu --metrics sm__cycles_elapsed.sum --cache-control none ./vector_add

We can also measure this behavior more directly. Nsight Compute can tell us how many bytes were moved from DRAM. Using the metric `dram__bytes.sum`. Our hypothesis here is that if the arrays `a` and `b` we are reading from are cached, the amount of total traffic to DRAM could be as little as 1/3 of the DRAM traffic used in the first iteration. (Since we're writing to the array `c`, we still need to have traffic to DRAM to update its values.) What do your results show?

In [ ]:
!nvcc -arch=native -o vector_add exercises/vector_add.cu; ncu --metrics dram__bytes.sum --cache-control none ./vector_add

If you like, you can also directly measure instead the metrics `dram__bytes_read.sum` and `dram__bytes_write.sum` to see the breakdown between the two traffic directions.

## Global Memory Operations

With the above picture of the memory hierarchy in our mind, we want to drill into the idea of how memory operations behave and what the performance implications are.

We can divide memory operations into two types, loads and stores, and we could further subdivide these into **non-caching loads** and **caching loads**. 

Focusing first on caching loads, which are the most important and most common case, this simply means that caching is enabled for this type of transaction, and this is the default behavior of the GPU -- the L2 cache is always enabled. When we load data from global memory, the load request doesn't necessarily immediately go to device memory. First we attempt to "hit" in L1 cache, to see if it's already present there. If the data is already there because it was recently accessed by a thread on that SM, we can improve processor performance because we don't need to go all the way back to DRAM. If the L1 query reveals the data is not there (a "miss") then we will check L2. Since all SMs talk to L2 cache, only one thread somewhere on the GPU needs to have recently accessed this data for it to be present in L2. If we miss in L1 and miss in L2, then we have to retrieve the data from device memory. We'll focus on loads from device memory for the moment, and not rely on benefits from L1 and L2. (These are generally not user managed anyway.) 

For caching loads, the **load granularity** is a 128-byte line. (That is, we cannot simply ask for one byte and receive one byte, we'll receive much more than that.)

For stores, the typical methodology is first to **invalidate** the cache line in L1 (the contents of that cache line are no longer valid), so that future accesses to L1 will miss. In terms of L2, L2 is a write-back cache, which means that the data being written on the store operation will update L2, and then at some future time L2 will update device memory.

There are also **non-caching loads**, which means that we always miss in L1. When we load some memory, we don't bother checking L1, we just go straight to L2. There may be reasons why you want to do this that we won't deeply delve into (for example, if your program is trying to "communicate" in L2 between threads in different SMs without wanting to worry about the L1 cache getting in the way; also, there are some corner cases where non-caching loads may result in higher performance). When using `nvcc` to compile, you would use the `-Xptxas -dlcm=cg` flag to force loads to be non-caching. The load granularity for this case would be 32 bytes.

### Load Operation

A load operation is a load instruction issued warp-wide (recall that a **warp** is a group of 32 threads that execute simultaneously). All memory instructions, like all other instructions, are issued to warps, not individual threads. This doesn't mean every thread in the warp is doing exactly the same thing; for example, they could be accessing different memory locations. Since each thread in a warp can access a different address, a typical case is the 32 threads in a warp accessing 32 different addresses. We need to retrieve all 32 locations in memory to satisfy the load operation.

The memory controller will take that pattern of addresses and determine which cache lines/sectors of device memory are needed. (Segments of DRAM are 32 bytes on NVIDIA GPUs and are called **sectors**; these are the fundamental units of load/store instructions, so you cannot simply write or read a single byte from DRAM's perspective.) These are then retrieved and deposited in cache and ultimately in registers.

#### Caching Load

We are going to look at several examples of caching loads, with the difference in each case being the difference in the pattern of addresses loaded by the warp.

In this first case, the loads from the warp are adjacent and sequential. That is, the warp requests 32 aligned, consecutive 4-byte words. In the figure below, each arrow points to an address loaded by a thread. A line of C code that might generate this:

```
int c = a[idx];
```

where `idx` is our canonical globally unique index in the grid of threads:

```
int idx = threadIdx.x + blockIdx.x * blockDim.x;
```

Since this indexing is sequential in a warp, each thread in a warp is accessing adjacent locations in the array.

In our hypothetical example, we'll access locations starting on a sector boundary. We're showing a 128-byte cache line for illustration (though you could imagine decomposing this into 32-byte blocks if we were comparing to this sectors in device memory).

This request happens to fit in a single cache line: we're requesting 32\*4 = 128 bytes from this warp, and we can service this in a single 128-byte cache line. Every byte requested is used, so this is a **bus utilization** of 100%, and we would refer to this is a perfect **coalescing** scenario. That is, the addresses requested by the threads are grouped together by the memory controller into cache lines/sectors, and perfect/ideal coalescing is when all addresses can be coalesced together into a single cache line.

![](images/caching_load_1.png)

For contrast, consider this contrived example:

```
int c = a[rand() % warpSize];
```

where we generate a random number that is constrained to fall within a warp (in this case, between 0 and 31). We've depicted below which threads accesses which locations in memory. Due to our contrived scenario, all of the requests happen fall in the same cache line/sector. So there is actually no difference from the perspective of the memory system: the memory controller understands that all addresses fall in the same line and can sort this out (we just have to ensure that the right threads get the right data at the end, in some sort of "swizzle" operation). Thus the bus utilization is again 100% and this is a perfectly coalesced load.

![](images/caching_load_2.png)

Let's now look at deviations from the 100% utilization case. In our first case, we access data with a constant offset.

```
int c = a[idx-2];
```

So for example, thread 0 in the warp is accessing index 30 instead of index 32; thread 1 is accessing index 31 instead of 33, etc. So while the accesses are adjacent, the accesses straddle a cache line/sector boundary. Now, these cache line/sector boundaries are fixed and determined by the architecture. So this operation is coalesced into two separate groups serviced by two cache lines (with most of the threads coalesced into the second line/sector). So now the memory controller is requesting 256 bytes instead of 128 bytes (we have to retrieve a full 128 byte cache line for the first two threads in the warp, even though we're only using 8 bytes). As a result, bus utilization is 50%.

This is a non-ideal scenario -- if this were the main operation in your kernel, you'd immediately see a substantial hit in performance. However, since we're thinking about memory accesses in terms of cached accesses, this means that we'll benefit from the fact that other warps are likely to be in play here. If we were doing this operation on a long vector, for example a vector add operation on two long vectors where every warp is accessing data at index `idx-2`, we'd benefit from the fact that there's warps to the left and right of the one we're looking at. Those other warps are loading data from the adjacent cache lines (the first and third in this image). So in the asymptotic limit, on average each warp is pulling in from memory approximately only one line: the cache will mostly fix the problem and give us something much closer to 100% utilization.

![](images/caching_load_3.png)

In this next case, every thread in the warp is retrieving the same location in memory. This access pattern is valid and can be serviced, and takes approximately the same time to service as if we had retrieved the entire cache line. We're pulling in 128 bytes when we only need 4, so the bus utilization is 4/128 = 3.125%. This is totally fine to do once in a while. However, if your code is dominated by this access pattern, your performance will likely be suboptimal and you may want to refactor your code or your data so you don't have this pattern occurring frequently.

```
int c = a[40];
```

![](images/caching_load_4.png)

Now let's look at the worst case. In this contrived example, we're reading a fully random location in the array.

```
int c = a[rand()];
```

While this is not especially realistic, there are occasionally patterns like this in the case of indirect indexing, e.g. `a[b[idx]]`. The addresses from each warp point basically anywhere in memory, and the worst case is when one thread in a warp accesses a cache line that no other thread is accessing -- that is, where every thread requires its own cache line load. This worst case pattern is again 4/128 = 3.125% utilization. More generally, when your warp is loading `N` cache lines, each of 128 bytes, then the bus utilization is `128 / (N * 128)`, and this worst case corresponds to `N = 32`.

![](images/caching_load_5.png)

#### Non-caching Load

Let's look at one more example. This is similar to the previous case, with

```
int c = a[rand()];
```

but let's look at the non-caching load case (e.g. we compiled with `nvcc -Xptxas -dlcm=cg`). When we bypass the L1 cache, the minimum transaction size becomes 32 bytes (the size of a memory sector) rather than the 128 byte cache line size, and we can improve overall performance as a result (we're retrieving less wasted memory). So now only `N*32` bytes move across the bus on a miss rather than `N*128` bytes, and so our bus utilization is `128 / (N * 32)`, or 12.5% in the worst case of `N = 32`.

![](images/non-caching_load.png)

#### Global Memory Optimization Guidelines

So, to sum up:

**Strive for perfect coalescing**. It is not a disaster for every access to be non-coalesced, but frequent non-coalesced accesses may limit performance. You should strive to write code (and be able to read it by inspection) that is perfectly coalesced. That is, warps should access memory addresses in a contiguous region.

**Memory bandwidth-bound codes must saturate the memory bus to achieve optimal performance**. In order to do this, we must have enough concurrent accesses to saturate the bus -- that is, we must have a sufficiently large number of threads launched. (There are other strategies like processing several elements per thread.)

**Use all the caches**. While L1 and L2 are not user-managed, there are other caches that can be opted into such as the constant cache, if you need that last few percent of performance.

## Exercise: Matrix Row and Column Sums

In the code [exercises/matrix_sums.cu](exercises/matrix_sums.cu), we've provided an outline of an application that sums either the rows or the columns of a matrix (one kernel is provided for each case). Your first task is simply to edit the source file to complete the kernels that calculate the row sums and column sums. If you complete the exercise correctly, you will see the following output:

```
row sums correct!
column sums correct!
```

In [ ]:
!nvcc -arch=native -o matrix_sums exercises/matrix_sums.cu; ./matrix_sums

If you get stuck, you can consult an example solution in [solutions/matrix_sums.cu](solutions/matrix_sums.cu).

When you've got the code working correctly, let's examine the performance of these kernels with Nsight Compute. In particular, pay attention to the kernel durations -- are they the same or different? Can you explain the result in terms of what we have learned about global memory performance?

In [ ]:
!ncu ./matrix_sums

Let's dig into this result.

In [ ]:
!ncu --metrics smsp__sass_average_data_bytes_per_sector_mem_global_op_ld.pct ./matrix_sums

This metric, `smsp__sass_average_data_bytes_per_sector_mem_global_op_ld.pct`, measures the average percentage of bytes per sector used by the kernel relative to the total number of bytes requested. We can think of it as measuring **global load efficiency**. In the `row_sums` example, we are using 1/8 of the total bytes, or 4 bytes out of every (32-byte) sector, while in the `column_sums` example, we are using 32 bytes out of every 32 byte sector. In general, our goal is to maximize the number of bytes used per sector, as that is naturally a more efficient use of the memory subsystem.

If you like, you can also collect the metrics `l1tex__t_requests_pipe_lsu_mem_global_op_ld.sum`, the number of **requests** that were made to global memory, and `l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum`, the number of sectors of global memory that were loaded from or stored to. So the first one measures what the SMs asked for, while the second meaures what was actually transferred. When thought about this way, our goal is to maximize the number of sectors accessed per request. If we measure the ratio of transactions to requests, we want the ratio to be as high as possible. A higher ratio means we had more data transferred per request and a more effective bus utilization, so our memory accesses are higher efficiency.

So, does what happened in these two cases match your understanding of the kernel and the DRAM architecture?

## Exercise: Matrix Sums in Double Precision

Modify [exercises/matrix_sums.cu](exercises/matrix_sums.cu) to use a double precision matrix and double precision accumulation vector, but otherwise leave the code unchanged. We've typedef'ed `real` to make this easy. Collect the profile and global load efficiency metric again. What changed? Do the results make sense given that the size of a word is now 8 bytes instead of 4?

In [ ]:
!nvcc -arch=native -o matrix_sums exercises/matrix_sums.cu; ./matrix_sums

In [ ]:
!ncu ./matrix_sums

In [ ]:
!ncu --metrics smsp__sass_average_data_bytes_per_sector_mem_global_op_ld.pct ./matrix_sums

## Summary

What we learned in this module is:

- Global memory has an access pattern that likes coalesence
  - Access contiguous locations in memory with contiguous threads

Remember that this guideline is meant to guide your initial implementation of an algorithm, but you should always profile your code after writing it, and use the NVIDIA profiling tools to guide the optimization process.

## Further Study

[Optimization in-depth](http://on-demand.gputechconf.com/gtc/2013/presentations/S3466-Programming-Guidelines-GPU-Architecture.pdf)


[Analysis-Driven Optimization](http://on-demand.gputechconf.com/gtc/2012/presentations/S0514-GTC2012-GPU-Performance-Analysis.pdf)


[CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html)


[CUDA Tuning Guides](https://docs.nvidia.com/cuda/index.html#programming-guides) (Kepler/Maxwell/Pascal/Volta)

## Lab Materials

You can download this notebook using the `File > Download as > Notebook (.ipnyb)` menu item. Source code files can be downloaded from the `File > Download` menu item after opening them.